<a href="https://colab.research.google.com/github/yueguo1997/SNA_recommender_system/blob/main/SNA_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
from google.colab import drive
import os
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
drive.mount('/content/drive')

# Data preparation

In [5]:
cur_path = "/content/drive/My Drive/SNA final project/"
os.chdir(cur_path)
!pwd

/content/drive/My Drive/SNA final project


In [37]:
# Reading in datasets/book1.csv
df = pd.read_csv('bonanza_buyer_item_seller_id_R_2.txt', sep='::', header=None, names=['buyer', 'item', 'seller', 'timestamp'])

# Printing out the head of the dataset
# split the data into train and test
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

<ipython-input-37-8e4ad435eb4a>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('bonanza_buyer_item_seller_id_R_2.txt', sep='::', header=None, names=['buyer', 'item', 'seller', 'timestamp'])


# Create Graph

In [38]:
import networkx as nx
import csv
G_train = nx.MultiGraph()
G_test = nx.MultiGraph()
# Add nodes for buyers, items, and sellers
for node_type in ['buyer', 'item', 'seller']:
    nodes1 = train_data[node_type].unique()
    nodes2 = train_data[node_type].unique()
    for node in nodes1:
        G_train.add_node(node, type=node_type)
    for node in nodes2:
        G_test.add_node(node, type=node_type)

# Add edges for interactions
for _, row in train_data.iterrows():
    buyer = row['buyer']
    item = row['item']
    seller = row['seller']
    G_train.add_edge(buyer, item, type='buys')
    G_train.add_edge(item, seller, type='sold_by')
for _, row in test_data.iterrows():
    buyer = row['buyer']
    item = row['item']
    seller = row['seller']
    G_test.add_edge(buyer, item, type='buys')
    G_test.add_edge(item, seller, type='sold_by')

In [40]:
# Print some statistics about the graph
print("Number of nodes on train:", G_train.number_of_nodes())
print("Number of edges on train:", G_train.number_of_edges())
print("Number of unique edge types on train:", len(set(nx.get_edge_attributes(G_train, 'type').values())))

print("Number of nodes on test:", G_test.number_of_nodes())
print("Number of edges on test:", G_test.number_of_edges())
print("Number of unique edge types on test:", len(set(nx.get_edge_attributes(G_test, 'type').values())))

Number of nodes on train: 8501
Number of edges on train: 62560
Number of unique edge types on train: 2
Number of nodes on test: 8922
Number of edges on test: 15640
Number of unique edge types on test: 2


# Node embedding vased on deepwalk

In [18]:
from gensim.models import Word2Vec
import random


In [19]:
def random_walks(graph, num_walks, walk_length):
    walks = []
    nodes = list(graph.nodes())
    for _ in range(num_walks):
        random.shuffle(nodes)
        for node in nodes:
            walk = [node]
            while len(walk) < walk_length:
                neighbors = list(graph.neighbors(walk[-1]))
                if neighbors:
                    walk.append(random.choice(neighbors))
                else:
                    break
            walks.append([str(node) for node in walk])
    return walks

In [43]:
# Learn node embeddings using DeepWalk
walks_train = random_walks(G_train, num_walks=10, walk_length=80)
model = Word2Vec(walks_train, vector_size=128, window=5, min_count=0, sg=1, workers=4)

# Get the embeddings for the nodes
node_embeddings = {str(node): model.wv[str(node)] for node in G_train.nodes()}

In [44]:
len(node_embeddings)

8501

# Model training